In [1]:
import os
import cv2
import glob
import logging
import numpy as np
from skimage import feature
from sklearn.svm import SVC 

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
source_dir='test'
dest_dir='test_cropped'

In [4]:
def mkdir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
            logging.debug(f"Folder '{path}' created successfully.")
        else:
            raise RuntimeError(f"Folder '{path}' already exists.")
    except Exception as e:
        logging.error(f"Error creating folder '{path}': {e} You are going to modify an existing file")
        raise

In [5]:
def face_extractor(source_dir, dest_dir, ext='jpg'):
    mkdir(dest_dir)
    haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') 
    
    for pimg in glob.glob(f"{source_dir}/*.{ext}"):
        img=cv2.imread(pimg)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        faces_rect = haar_cascade.detectMultiScale(gray_img, 1.2, 5) 
        
        if faces_rect is ():
            logging.debug('No face detected')
            continue
        
        for (x, y, w, h) in faces_rect: 
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2) 
            cropped_face = gray_img[y:y+h, x:x+w]
            img_name = os.path.splitext(os.path.basename(pimg))[0]
            cv2.imwrite(f'{dest_dir}/{img_name}_cropped.jpg', cropped_face) 

In [6]:
face_extractor(source_dir, dest_dir)

ERROR:root:Error creating folder 'test_cropped': Folder 'test_cropped' already exists. You are going to modify an existing file


RuntimeError: Folder 'test_cropped' already exists.

In [7]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
                                 bins=np.arange(0, self.numPoints + 3),
                                 range=(0, self.numPoints + 2))
        
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

In [8]:
def pattern_extractor(desc: LocalBinaryPatterns, source_dir='./test_cropped', ext='jpg'):
    data = []
    labels = []
    
    for pimg in glob.glob(f"{source_dir}/*.{ext}"):
        img = cv2.imread(pimg)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hist = desc.describe(gray_img)
        labels.append(os.path.basename(pimg).split('_')[1])
        data.append(hist)
        
    return data, labels

In [9]:
data, labels = pattern_extractor(LocalBinaryPatterns(24, 8))

KeyboardInterrupt: 

In [ ]:
model = SVC(kernel='rbf', random_state=42)
model.fit(data, labels)

In [ ]:
def gender_detector(desc: LocalBinaryPatterns, source, ext='jpg'):
    if os.path.isdir(source):
        logging.debug(f"{source} is a directory full of image.")
        
        data = []
        for pimg in glob.glob(f"{source_dir}/*.{ext}"):
            img = cv2.imread(pimg)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            hist = desc.describe(gray_img)
            data.append(hist)

    elif os.path.isfile(source):
        logging.debug(f"{source} is a single image.")

    else:
        print(f"{source} does not exist.")

In [22]:
source_dir="C:\\Users\\aerol\\Desktop\\gender_detection\\7_0_3_20170119144311624"
ext="jpg"

In [23]:
for pimg in glob.glob(f"{source_dir}/*.{ext}"):
        img = cv2.imread(pimg)
        cv2.imshow("image", img)
        cv2.waitKey(0)  # Wait indefinitely for a key press
        cv2.destroyAllWindows()